In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import json

from openai import OpenAI

from elections import constants
from elections.utils import escape_quotes_within_citation 
from elections.prompts.templates import sentiment_template

In [23]:
client = OpenAI()

In [ ]:
sentiment = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Say this is a test"}],
)

sentiment.choices[0].message.content

In [26]:
print(
    sentiment_template.SYSTEM_PROMPT.format(
        politicians=constants.POLITICIANS,
        json_format=sentiment_template.JSON_FORMAT
    )
)

TASK
You will be provided with a document delimited by triple quotes and a question. Your task is to answer the question using only the provided document and to cite the passage(s) of the document used to answer the question. If the document does not contain the information needed to answer this question, then simply write: {}.If an answer to the question is provided, it must be in the following JSON format, the terms surrounded by < > are variables: [{"name": "<politician name>","score": <the score you determined>,"citations": [{"citation": "<citation 1>","positive_proba": <probability>}]}]

PROCESS
Take your time to answer the question and go through the following steps:
1.	identify all the politicians in the article who are present in this list: ['Pedro Nuno Santos', 'Luís Montenegro', 'André Ventura', 'Rui Rocha', 'Mariana Mortágua', 'Paulo Raimundo', 'Inês de Sousa Real', 'Rui Tavares']
2.	for each politician identify the parts of the article that refer to them
3.	for each part de

In [18]:
print(sentiment_template.USER_PROMPT)

ARTICLE:
"""{article}"""  

QUESTION: 
Only provide a JSON for your analysis and nothing more. The politicians score goes from 1 to 5 (with 1 being extremely bad). If the opinion is neutral provide a 3, if the article is only factual and does not have an opinion then the score should be None. 



In [19]:
import sqlite3
import pandas as pd
from elections import constants

engine = sqlite3.connect(constants.NEWS_DB)
df = pd.read_sql("SELECT * FROM articles LIMIT 5", engine)
df

,title,description,pubdate,publisher,url,summary,keywords,text,creation_datetime,query
0,Campanha a todo o gás e a prometedora entrevis...,Campanha a todo o gás e a prometedora entrevis...,2023-12-01T08:00:00,SIC Notícias,https://news.google.com/rss/articles/CBMilwFod...,"Para Luís Montenegro, é ""indiferente"" quem suc...","[""costa"", ""sic"", ""luís"", ""prometedora"", ""antón...","Para Luís Montenegro, é ""indiferente"" quem suc...",2024-02-11T17:26:45.678832,'Luis Montenegro'
1,"""Estamos mais preparados do que quaisquer outr...","""Estamos mais preparados do que quaisquer outr...",2023-12-02T08:00:00,Expresso,https://news.google.com/rss/articles/CBMijwFod...,O presidente do PSD defendeu este sábado que o...,"[""governar"", ""quaisquer"", ""programa"", ""preside...",O presidente do PSD defendeu este sábado que o...,2024-02-11T17:26:45.678832,'Luis Montenegro'
2,Luís Montenegro: “Qualquer pessoa tem o direit...,Luís Montenegro: “Qualquer pessoa tem o direit...,2024-02-10T15:30:00,Expresso,https://news.google.com/rss/articles/CBMiwgFod...,O cabeça de lista por Lisboa da Aliança Democr...,"[""sá"", ""tentou"", ""utilizar"", ""sic"", ""socialdem...",O cabeça de lista por Lisboa da Aliança Democr...,2024-02-11T17:40:25.902188,'Luis Montenegro'
3,Luís Montenegro nega “falta de comparência do ...,Luís Montenegro nega “falta de comparência do ...,2024-02-03T08:15:00,SIC Notícias,https://news.google.com/rss/articles/CBMioQFod...,"Para já, Luís Montenegro enfrenta a ameaça do ...","[""política"", ""passos"", ""casa"", ""espinho"", ""luí...",Aos 9 anos ia a pé para a escola e já “tinha a...,2024-02-11T17:40:25.902188,'Luis Montenegro'
4,Luís Montenegro recusa ter medo de debates pol...,Luís Montenegro recusa ter medo de debates pol...,2024-02-08T19:55:48,Público,https://news.google.com/rss/articles/CBMiaGh0d...,Líder social-democrata não responde a repto de...,"[""medo"", ""luís"", ""políticos"", ""ad"", ""debate"", ...",Líder social-democrata não responde a repto de...,2024-02-11T17:40:25.902188,'Luis Montenegro'


In [25]:
print(
    df["text"][0]
)

Para Luís Montenegro, é "indiferente" quem sucederá a António Costa na liderança do Partido Socialista, mas não para Pedro Delgado Alves e José Eduardo Martins, neste Antes Pelo Contrário em podcast. Em entrevista à SIC, o líder do PSD lembrou que tanto José Luís Carneiro como Pedro Nuno Santos "fizeram parte dos governos de António Costa" que, segundo a sua avaliação, tiveram "muito pouco investimento público". Mas é ao antigo ministro das Infraestruturas que dirigiu o maior ataque: "Está associado a três das maiores trapalhadas que este governo trouxe para a política portuguesa". O Antes Pelo Contrário foi emitido a 30 de novembro na SIC Notícias.


In [52]:
system_prompt = sentiment_template.SYSTEM_PROMPT.format(
    politicians=constants.POLITICIANS,
    json_format=sentiment_template.JSON_FORMAT
)

user_prompts = df["text"].apply(
    lambda x: sentiment_template.USER_PROMPT.format(article=x)
)

sentiment = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompts[0]}],
)

sentiment.choices[0].message.content

'[{"name": "Luís Montenegro", "score": 3, "citations": []}, {"name": "Pedro Nuno Santos", "score": 2, "citations": [{"citation": "O líder do PSD lembrou que tanto José Luís Carneiro como Pedro Nuno Santos "fizeram parte dos governos de António Costa" que, segundo a sua avaliação, tiveram "muito pouco investimento público".", "positive_proba": 0.4}]}]'

In [107]:
from json_repair import repair_json

def process_sentiment(resp):
    sentiment = resp.choices[0].message.content
    try:
        sentiment = json.loads(sentiment)
    except json.JSONDecodeError:
        sentiment = escape_quotes_within_citation(sentiment)
    sentiment = repair_json(sentiment, return_objects=True)
    sentiment = [
        opinion if opinion["citations"] 
        else {"name": opinion["name"], "score": "None", "citations": []}
        for opinion in sentiment
    ]
    return sentiment

In [108]:
process_sentiment(sentiment)

[{'name': 'Luís Montenegro', 'score': 'None', 'citations': []},
 {'name': 'Pedro Nuno Santos',
  'score': 2,
  'citations': [{'citation': 'O líder do PSD lembrou que tanto José Luís Carneiro como Pedro Nuno Santos "fizeram parte dos governos de António Costa" que, segundo a sua avaliação, tiveram "muito pouco investimento público".',
    'positive_proba': 0.4}]}]

[{'name': 'Luís Montenegro', 'score': 'None', 'citations': []},
 {'name': 'Pedro Nuno Santos',
  'score': 2,
  'citations': [{'citation': 'O líder do PSD lembrou que tanto José Luís Carneiro como Pedro Nuno Santos ',
    'Costa': 'que',
    'muito pouco investimento público': ', ',
    'positive_proba': 0.4}]}]

In [57]:
json.loads(good_json_string)

[{'name': 'Luís Montenegro', 'score': 3, 'citations': []},
 {'name': 'Pedro Nuno Santos',
  'score': 2,
  'citations': [{'citation': 'O líder do PSD lembrou que tanto José Luís Carneiro como Pedro Nuno Santos ',
    'Costa': 'que',
    'muito pouco investimento público': ', ',
    'positive_proba': 0.4}]}]

In [53]:
import json
json.loads(sentiment.choices[0].message.content)

JSONDecodeError: Expecting ',' delimiter: line 1 column 206 (char 205)

In [ ]:
print(
    sentiment_template.SYSTEM_PROMPT.format(
        politicians=constants.POLITICIANS,
        json_format=sentiment_template.JSON_FORMAT
    )
)

In [ ]:
print(sentiment.choices[0].message.content)